# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vacation_df = pd.read_csv("city_data.csv", index_col=False)
vacation_df.reset_index(drop=True, inplace=True)
vacation_df


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
0,jamestown,42.10,-79.24,57.20,82.0,1.0,6.93
1,souillac,-20.52,57.52,69.80,73.0,20.0,6.93
2,udachnyy,66.42,112.40,51.80,40.0,0.0,6.71
3,vila franca do campo,37.72,-25.43,64.40,72.0,40.0,13.87
4,bambous virieux,-20.34,57.76,73.99,73.0,20.0,6.93
...,...,...,...,...,...,...,...
547,abha,18.22,42.51,64.40,27.0,71.0,7.52
548,tecolutla,20.48,-97.02,77.65,76.0,85.0,4.63
549,southbridge,42.08,-72.03,60.01,82.0,0.0,5.03
550,lorengau,-2.02,147.27,86.00,58.0,45.0,6.67


In [4]:
# Store latitude and longitude in locations
locations = vacation_df[["Latitude", "Longitude"]]
humidity = vacation_df["Humidity"].astype(float)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = vacation_df[['Latitude', 'Longitude']].astype(float)

# Convert Humidity to float and store
humidity = vacation_df['Humidity'].astype(float)
print(len(locations))
print(len(humidity))

552
552


In [6]:
# Plot Heatmap
fig = gmaps.figure()

# customize the size of the figure
fig_layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=fig_layout)

# heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# add layer
fig.add_layer(heat_layer)

# display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Weather conditions For New Data Frame
conditions_df = vacation_df.loc[(vacation_df["Max Temp"]>70) & (vacation_df["Max Temp"]<85) & (vacation_df["Humidity"]<50) & (vacation_df["Cloudiness"]<50)]
conditions_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
9,rio verde de mato grosso,-18.92,-54.84,79.95,43.0,33.0,12.66
29,yulara,-25.24,130.99,78.80,14.0,0.0,6.93
42,alice springs,-23.70,133.88,77.00,15.0,0.0,9.17
56,sinjar,36.32,41.88,75.06,42.0,0.0,2.93
96,saint-philippe,-21.36,55.77,73.40,43.0,43.0,9.17
126,santiago del estero,-27.80,-64.26,72.00,43.0,9.0,7.67
146,airai,-8.93,125.41,80.11,43.0,0.0,6.64
160,saint-joseph,-21.37,55.62,73.40,43.0,45.0,9.17
171,saint george,37.10,-113.58,82.40,17.0,1.0,3.36
196,shache,38.42,77.24,77.05,17.0,0.0,1.86


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = conditions_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
9,rio verde de mato grosso,-18.92,-54.84,79.95,43.0,33.0,12.66,
29,yulara,-25.24,130.99,78.80,14.0,0.0,6.93,
42,alice springs,-23.70,133.88,77.00,15.0,0.0,9.17,
56,sinjar,36.32,41.88,75.06,42.0,0.0,2.93,
96,saint-philippe,-21.36,55.77,73.40,43.0,43.0,9.17,


In [10]:
#param that updates with each iteration 
params = {"key": g_key,
         "type": "lodging",
         "radius": 5000}

# Loop through the data rame and run a lat long 
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # update address key value
    params['location'] = f"{latitude},{longitude}"

    # make request
    hotels = requests.get(base_url, params=params).json()
    response = hotels['results']
    
    try:
               
        hotel_df.loc[index, "Hotel Name"] = response[0]['name']
        
    except (KeyError, IndexError):
        
        hotel_df.loc[index, "Hotel Name"] = ""

hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != ""), :]
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

NameError: name 'fig' is not defined